In [1]:
import warnings

import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from fairfl.utils.read_data import DatasetUtils
import matplotlib.pyplot as plt

from aif360.algorithms.preprocessing import Reweighing
from aif360.datasets import BinaryLabelDataset
from fairfl.model.model import ACSIncomeModel
from fairfl.metrics.bias_metrics import BiasType
from fairfl.utils.constants import states

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


In [3]:
utils = DatasetUtils()
bias_type = BiasType.race

model = ACSIncomeModel(bias_type=bias_type)
states_selected = ["MT", "NH", "ND", "ME", "SD"]

In [4]:
train = pd.read_csv("../dataset/train_preprocessed.csv")
test = pd.read_csv("../dataset/test_preprocessed.csv")

In [5]:
train_selected_states = utils.get_selected_states(train, states_selected, False).reset_index(drop=True)
test_selected_states = utils.get_selected_states(test, states_selected, False).reset_index(drop=True)

In [6]:
train_dataset = BinaryLabelDataset(df=train_selected_states,
                                   favorable_label=1, unfavorable_label=0,
                                   label_names=['PINCP'], protected_attribute_names=['RAC1P'],
                                   privileged_protected_attributes=[[1]],
                                   unprivileged_protected_attributes=[[0]])


In [7]:
privileged_groups = [{'RAC1P': 1}]
unprivileged_groups = [{'RAC1P': 0}]

RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(train_dataset)

In [8]:
train_transformed = dataset_transf_train.convert_to_dataframe()[0]

In [13]:
states_result_dfs = list()
states_selected = ["MT", "NH", "ND", "ME", "SD"]
for state in states_selected:
    res = model.print_model_metrics_for_state(train_transformed, test, state)
    states_result_dfs.append(res)

In [14]:
pd.concat(states_result_dfs)

,state,egitim_ornek_sayisi,test_ornek_sayisi,f1_score,disparate_impact,equal_opportunity_diff,statistical_parity
0,MT,3725,1738,0.581395,0.424452,0.0,0.287774
0,NH,5584,2382,0.742290,1.174968,-0.5,-0.058323
0,ND,3129,1326,0.670247,1.261442,-1.0,-0.069718
0,ME,4934,2068,0.629358,1.792115,0.0,-0.108016
0,SD,3438,1461,0.517720,2.495856,-0.5,-0.115066


In [10]:
pd.concat(states_result_dfs)

,state,egitim_ornek_sayisi,test_ornek_sayisi,f1_score,disparate_impact,equal_opportunity_diff,statistical_parity
0,MT,3725,1738,0.581395,2.355978,0.0,0.287774
0,NH,5584,2382,0.742290,0.851087,-0.5,-0.058323
0,ND,3129,1326,0.670247,0.792744,-1.0,-0.069718
0,ME,4934,2068,0.629358,0.558000,0.0,-0.108016
0,SD,3438,1461,0.517720,0.400664,-0.5,-0.115066


In [11]:
pd.concat(states_result_dfs).to_excel(f"../results/{bias_type}_bias_results.xlsx", index=False)